In [ ]:
# For string similarity
# !pip3 install py_stringsimjoin

In [1]:
# Python3 program for a word frequency counter after crawling/scraping a web-page
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import urllib
import urllib.request
from lxml import etree
import re
import sqlite3

# County Listings

In [2]:
def webdoc(url):

  # requests to fetch the HTML content of the website
  source_code = requests.get(url).text

  # BeautifulSoup to parse the HTML content
  return BeautifulSoup(source_code, 'lxml')

In [3]:
def get_county_list(soup):

  # find_all method to find the desired HTML elements
  a_tags = soup.find_all('a', href=True)
  county_list = []

  # Loop through the HTML elements
  for tag in a_tags:

    # Only get the specific text
    if str(tag).find('/secondary-schools-in-ireland/') >= 0:

      # append the text with the https to make it public url
      if str(tag).find('https://www.schooldays.ie') >= 0:
        pass
      else:
        county_list.append('https://www.schooldays.ie' + tag['href'])

  # Reduce the duplicates
  county_set = set(county_list)
  # print(county_list)
  # print(county_set)

  # Convert back to list so as to iter over it
  county_list = list(county_set)

  return county_list

In [4]:
url = "https://www.schooldays.ie/articles/secondary-Schools-in-Ireland-by-County"
# get the soup object
soup = webdoc(url)

# Get all the urls for the counties on the page
county_list = get_county_list(soup)

In [5]:
county_list[0:5]

['https://www.schooldays.ie/secondary-schools-in-ireland/Dublin-24',
 'https://www.schooldays.ie/secondary-schools-in-ireland/Sligo',
 'https://www.schooldays.ie/secondary-schools-in-ireland/Kerry',
 'https://www.schooldays.ie/secondary-schools-in-ireland/Dublin-3',
 'https://www.schooldays.ie/secondary-schools-in-ireland/Dublin-16']

# Schools from each county

Function lists all the schools on the site

In [6]:
def get_secondaryschools(soup):

  # find_all method to find the desired HTML elements
  a_tags = soup.find_all('a', href=True)
  secondary_school_list = []

  # Loop through the HTML elements
  for tag in a_tags:

    # Only specific text
    if str(tag).find('/school/') >= 0:

      # Further only specific text
      if str(tag).find('-rollnumber-') >= 0:

        school_name = str(tag['href'])
        school_name = school_name.replace("/school/", "/sch/")
        secondary_school_list.append('https://www.schooldays.ie' + school_name + '/college-progression')
        # print(str(tag))

  # Reduce the duplicates
  secondary_school_set = set(secondary_school_list)
  # print(secondary_school_list)
  # print(secondary_school_set)
  # Convert back to list
  secondary_school_list = list(secondary_school_set)

  return secondary_school_list

In [7]:
# initialise the list
secondary_schools = []

for link in county_list:

  # get the soup object
  soup = webdoc(link)

  secondary_schools = secondary_schools + get_secondaryschools(soup)

In [8]:
secondary_schools[0:5]

['https://www.schooldays.ie/sch/st-marks-community-school-rollnumber-91332O/college-progression',
 'https://www.schooldays.ie/sch/st-aidans-community-school-rollnumber-91338D/college-progression',
 'https://www.schooldays.ie/sch/jobstown-community-college-rollnumber-70141N/college-progression',
 'https://www.schooldays.ie/sch/killinarden-community-school-rollnumber-91337B/college-progression',
 'https://www.schooldays.ie/sch/firhouse-educate-together-secondary-school-rollnumber-68307J/college-progression']

# Get the college progession Stats now

In [9]:
url = 'https://www.schooldays.ie/sch/ennistymon-vocational-school-rollnumber-70840Q/college-progression'
soup = webdoc(url)
content = str(soup.find('table', {'class': 'sch1table'}).find('tr').find('td').get_text())
print(content)
print(len(content))
print('---------------------------')
# content = re.sub("(http[s]?\://\S+)|\n","",content)
# content.split('</b>  College places as follows:<br>')
# soup.select('table.sch1table br')[3].nextSibling

 Leaving Cert & College Numbers - 
notes on data sources 
Total sitting Leaving Cert 2023: 21.  College places as follows:UCC 1 , UL 3 , Atlantic TU 1 , TU Shannon 8 ,  Total of main colleges 13.
Total sitting Leaving Cert 2022: 29.  College places as follows:UL 3 , TU Dublin 1 , Atlantic TU 1 , TU Shannon 6 , National College of Ireland 1 ,  Total of main colleges 12.
Total sitting Leaving Cert 2021: 47.  College places as follows:TCD 1 , Uni of Galway 3 , UL 4 , TU Dublin 1 , Mary Immaculate College 1 , TU Shannon 7 ,  Total of main colleges 21.
Total sitting Leaving Cert 2020: 41.  College places as follows:UCC 2 , Uni of Galway 1 , UL 5 , AIT 3 , Cork IT 1 , Galway Mayo 7 , LIT 5 , Mary Immaculate College 1 ,  Total of main colleges 25.
Total sitting Leaving Cert 2019: 23.  College places as follows:Maynooth U 1 , UL <5 , AIT 3 , Cork IT 1 , Galway Mayo 2 , LIT 3 ,  Total of main colleges 12.5.
Total sitting Leaving Cert 2018: 22.  College places as follows:Uni of Galway 2 , Galway

In [ ]:
# import re
# temp = re.findall(r'\d+', ' 200 grams')
# int(temp[0])

200

# Testing Code

In [10]:
def create_df():
  df = pd.DataFrame(columns=[
  'School',
  'Roll Number',
  'Year',
  'Total sitting Leaving Cert',
  'DCU',
  'Maynooth U',
  'TU Dublin',
  'Letterkenny',
  'National College of Ireland',
  'Total of main colleges',
  'TCD',
  'AIT',
  'Dundalk',
  'Waterford',
  'Cork IT',
  'DIT',
  'Blanchardstown',
  'Carlow IT',
  'Galway Mayo',
  'Tallaght',
  'National College of Art & Design',
  'IADT',
  'LIT',
  'UL',
  'Sligo',
  'NUIG', 'Mary Immaculate College'])
  return df

In [13]:
def get_college_progressions_test(soup, idx):




  name_rollnumber = url.split("https://www.schooldays.ie/sch/")[-1] #extract the name from the url

  name = name_rollnumber.split("-rollnumber-")[0]
  rollnumber = name_rollnumber.split("-rollnumber-")[1].split("/")[0]
  print(f'Looking at school {name} Rollnumber : {rollnumber}')

  # get the contents of the table
  content = str(soup.find('table', {'class': 'sch1table'}).find('tr').find('td').get_text())

  list_of_adm = []

  # get a new data frame for saving the stats
  df = create_df()

  for row in content.splitlines():

    DCUCount = 0.0
    MaynoothUCount = 0.0
    TUDCount = 0.0
    LetterkenyCount = 0.0
    NCICount = 0.0
    Totalofmain = 0.0
    TCDCount = 0.0
    AITCount = 0.0
    DundalkCount = 0.0
    WaterfordCount = 0.0
    CorkCount = 0.0
    DITCount = 0.0
    BlanchCount = 0.0
    CarlowCount = 0.0
    GalwayCount = 0.0
    TallaghtCount = 0.0
    NCADCount = 0.0
    IADTCount = 0.0
    LITCount = 0.0
    ULCount = 0.0
    SligoCount = 0.0
    NUIGCount = 0.0
    MaryImmaculateCount = 0.0
    TotalsittingLC = 0.0
    Totalofmain = 0.0
    YearsittingLC = 0
    # specific text only
    if row.startswith('Total sitting Leaving Cert'):

      # convert to string type
      string = str(row)
      # print(row)
      # print(string[0:26])
      YearsittingLC = int(string[27:31])

      # for few schools the year's totals are
      TotalsittingLC_str = string[32:37].replace('?','').strip()
      TotalsittingLC_str = TotalsittingLC_str.replace('<','')
      if TotalsittingLC_str[-1] == '.':
          TotalsittingLC_str = TotalsittingLC_str[:-1]

      # check if the string is null
      if len(TotalsittingLC_str) == 0:
        TotalsittingLC = 0.0
      else:
        TotalsittingLC = float(TotalsittingLC_str)

      #
      list_of_adm = [member.strip() for member in string[64:].split(', ')]
      # print(list_of_adm)
      for collegestat in list_of_adm:

        # strip the spaces at the ends
        collegestat = collegestat.strip()

        # find space from right
        if len(collegestat) > 0:
        # if collegestat.rindex(' ') > 0:
          key = collegestat[0:collegestat.rindex(' ')]
          value_str = collegestat[collegestat.rindex(' '):].replace('<','')
          if value_str[-1] == '.':
            value_str = value_str[:-1]
          value = float(value_str)
        # else:
          # key = 'Not Defined'
          # value = 0.0
        else:
            key = 'Not Defined'
            value = 0.0
        # print(f' Key : {key} , Value : {value}')
        if key == 'DCU':
          DCUCount = value
        if key == 'Maynooth U':
          MaynoothUCount = value
        if key == 'TU Dublin':
          TUDCount = value
        if key == 'Letterkenny':
          LetterkenyCount = value
        if key == 'National College of Ireland':
          NCICount = value
        if key == 'Total of main colleges':
          Totalofmain = value
        if key == 'TCD':
          TCDCount = value
          print("TCD ------------------")
        if key == 'AIT':
          AITCount = value
        if key == 'Dundalk':
          DundalkCount = value
        if key == 'Waterford':
          WaterfordCount = value
        if key == 'Cork IT':
          CorkCount = value
        if key == 'DIT':
          DITCount = value
        if key == 'Blanchardstown':
          BlanchCount = value
        if key == 'Carlow IT':
          CarlowCount = value
        if key == 'Galway Mayo':
          GalwayCount = value
        if key == 'Tallaght':
          TallaghtCount = value
        if key == 'National College of Art & Design':
          NCADCount = value
        if key == 'IADT':
          IADTCount = value
        if key == 'LIT':
          LITCount = value
        if key == 'UL':
          ULCount = value
        if key == 'Sligo':
          SligoCount = value
        if key == 'NUIG':
          NUIGCount = value
          print("NUIG ------------>>>>")
        if key == 'Mary Immaculate College':
          MaryImmaculateCount = value

      # for every row read, append it to the df
      # row = pd.DataFrame({'DCU': DCUCount, 'Maynooth U': MaynoothUCount,
      row_dict = {'DCU': DCUCount, 'Maynooth U': MaynoothUCount,
      # schoolstats_df.append({'DCU': DCUCount, 'Maynooth U': MaynoothUCount,
                          'TCD Dublin': TUDCount, 'Letterkeny':LetterkenyCount,
                          'National College of Ireland': NCICount, 'TCD': TCDCount,
                          'AIT': AITCount, 'Dundalk': DundalkCount,
                          'Waterford': WaterfordCount, 'Cork IT': CorkCount,
                          'DIT': DITCount, 'Blanchardstown': BlanchCount,
                          'Carlow IT': CarlowCount, 'Galway Mayo': GalwayCount,
                          'Tallaght': TallaghtCount,
                          'National College of Art & Design': NCADCount,
                          'IADT': IADTCount, 'LIT': LITCount, 'UL': ULCount,
                          'Sligo': SligoCount, 'NUIG': NUIGCount,
                          'Mary Immaculate College': MaryImmaculateCount,
                          'Total sitting Leaving Cert': TotalsittingLC,
                          'Year': YearsittingLC,
                          'Total of main colleges': Totalofmain,
                          'School': name, 'Roll Number': rollnumber
                          }
      # print(row_dict)
      row_dict = {k:[v] for k,v in row_dict.items()}
      # print(row_dict)
      row = pd.DataFrame(row_dict)
      # print(row)
      df = df.append(row, ignore_index=True)

  return df

In [14]:
crawl_list = ['https://www.schooldays.ie/sch/ennistymon-vocational-school-rollnumber-70840Q/college-progression']
# for url in crawl_list:
for index, url in enumerate(crawl_list):
  # url = page.geturl()
  soup = webdoc(url)
  # soup = webdoc(school)

  df = get_college_progressions_test(soup, index+1)
  df = df.fillna(0)

Looking at school ennistymon-vocational-school Rollnumber : 70840Q


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# print(df.info())
# print(df.head())

In [18]:
def get_college_progressions(soup, idx):


  DCUCount = 0.0
  MaynoothUCount = 0.0
  TUDCount = 0.0
  LetterkenyCount = 0.0
  NCICount = 0.0
  Totalofmain = 0.0
  TCDCount = 0.0
  AITCount = 0.0
  DundalkCount = 0.0
  WaterfordCount = 0.0
  CorkCount = 0.0
  DITCount = 0.0
  BlanchCount = 0.0
  CarlowCount = 0.0
  GalwayCount = 0.0
  TallaghtCount = 0.0
  NCADCount = 0.0
  IADTCount = 0.0
  LITCount = 0.0
  ULCount = 0.0
  SligoCount = 0.0
  NUIGCount = 0.0
  MaryImmaculateCount = 0.0
  TotalsittingLC = 0.0
  Totalofmain = 0.0
  YearsittingLC = 0

  name_rollnumber = url.split("https://www.schooldays.ie/sch/")[-1] #extract the name from the url

  name = name_rollnumber.split("-rollnumber-")[0]
  rollnumber = name_rollnumber.split("-rollnumber-")[1].split("/")[0]
  print(f'{idx}. Looking at school {name} Rollnumber : {rollnumber}')

  # get the contents of the table
  content = str(soup.find('table', {'class': 'sch1table'}).find('tr').find('td').get_text())

  list_of_adm = []

  # get a new data frame for saving the stats
  df = create_df()

  for row in content.splitlines():

    # specific text only
    if row.startswith('Total sitting Leaving Cert'):

      # convert to string type
      string = str(row)

      YearsittingLC = int(string[27:31])

      # for few schools the year's totals are
      TotalsittingLC_str = string[32:37].replace('?','').strip()
      TotalsittingLC_str = TotalsittingLC_str.replace('<','')
      if TotalsittingLC_str[-1] == '.':
          TotalsittingLC_str = TotalsittingLC_str[:-1]

      # check if the string is null
      if len(TotalsittingLC_str) == 0:
        TotalsittingLC = 0.0
      else:
        TotalsittingLC = float(TotalsittingLC_str)

      #
      list_of_adm = [member.strip() for member in string[64:].split(', ')]

      for collegestat in list_of_adm:

        # strip the spaces at the ends
        collegestat = collegestat.strip()

        # find space from right
        if len(collegestat) > 0:

          key = collegestat[0:collegestat.rindex(' ')]
          value_str = collegestat[collegestat.rindex(' '):].replace('<','')
          if value_str[-1] == '.':
            value_str = value_str[:-1]
          value = float(value_str)

        else:
            key = 'Not Defined'
            value = 0.0


        if key == 'DCU':
          DCUCount = value
        if key == 'Maynooth U':
          MaynoothUCount = value
        if key == 'TU Dublin':
          TUDCount = value
        if key == 'Letterkenny':
          LetterkenyCount = value
        if key == 'National College of Ireland':
          NCICount = value
        if key == 'Total of main colleges':
          Totalofmain = value
        if key == 'TCD':
          TCDCount = value
        if key == 'AIT':
          AITCount = value
        if key == 'Dundalk':
          DundalkCount = value
        if key == 'Waterford':
          WaterfordCount = value
        if key == 'Cork IT':
          CorkCount = value
        if key == 'DIT':
          DITCount = value
        if key == 'Blanchardstown':
          BlanchCount = value
        if key == 'Carlow IT':
          CarlowCount = value
        if key == 'Galway Mayo':
          GalwayCount = value
        if key == 'Tallaght':
          TallaghtCount = value
        if key == 'National College of Art & Design':
          NCADCount = value
        if key == 'IADT':
          IADTCount = value
        if key == 'LIT':
          LITCount = value
        if key == 'UL':
          ULCount = value
        if key == 'Sligo':
          SligoCount = value
        if key == 'NUIG':
          NUIGCount = value
        if key == 'Mary Immaculate College':
          MaryImmaculateCount = value

      # for every row read, append it to the df
      row_dict = {'DCU': DCUCount, 'Maynooth U': MaynoothUCount,
                          'TCD Dublin': TUDCount, 'Letterkeny':LetterkenyCount,
                          'National College of Ireland': NCICount, 'TCD': TCDCount,
                          'AIT': AITCount, 'Dundalk': DundalkCount,
                          'Waterford': WaterfordCount, 'Cork IT': CorkCount,
                          'DIT': DITCount, 'Blanchardstown': BlanchCount,
                          'Carlow IT': CarlowCount, 'Galway Mayo': GalwayCount,
                          'Tallaght': TallaghtCount,
                          'National College of Art & Design': NCADCount,
                          'IADT': IADTCount, 'LIT': LITCount, 'UL': ULCount,
                          'Sligo': SligoCount, 'NUIG': NUIGCount,
                         'Mary Immaculate College': MaryImmaculateCount,
                          'Total sitting Leaving Cert': TotalsittingLC,
                          'Year': YearsittingLC,
                          'Total of main colleges': Totalofmain,
                          'School': name, 'Roll Number': rollnumber
                          }

      # print(row_dict)
      row_dict = {k:[v] for k,v in row_dict.items()}
      # print(row_dict)
      row = pd.DataFrame(row_dict)
      # print(row)
      df = df.append(row, ignore_index=True)
      # schoolstats_df = pd.concat([schoolstats_df,row],join='inner')

  return df

In [21]:
# these are the actual pages with college stats
crawl_list = secondary_schools
print(f'Total number of schools to be crawled : {len(crawl_list)}')

# for url in crawl_list:
for index, url in enumerate(crawl_list):
  # url = page.geturl()
  soup = webdoc(url)
  # soup = webdoc(school)

  df = df.append(get_college_progressions(soup, index+1), ignore_index=True)
  df = df.fillna(0)

Total number of schools to be crawled : 847


NameError: name 'df' is not defined

In [22]:
df.to_csv('out.csv')

NameError: name 'df' is not defined

In [ ]:
print(df.shape)
print(df.head)

https://machinelearningmastery.com/web-crawling-in-python/

https://medium.com/data-analysis-enthusiast/how-to-write-a-web-crawler-in-python-f77a95a27a70

In [ ]:
# dataset is availabe in the google drive, hence need to mount the google drive to load the dataset
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# load dataset into memory
df.to_csv("/content/gdrive/MyDrive/MachineLearning/data/school_progression.csv")

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
G = nx.from_pandas_edgelist(df, source='School', target=['TCD Dublin','Letterkeny'])

In [ ]:
G.add_nodes_from(nodes_for_adding=df.ID.tolist())

In [ ]:
# import itertools.combinations as comb

edges = set()
for col in df:
    for _, data in df.groupby(col):
        edges.update(comb(data.index, 2))

G = nx.Graph()
G.add_nodes_from(df.index)
G.add_edges_from(edges)

ModuleNotFoundError: ignored